# DataFrame structure
Compiling the duration and time stamp data of Wake, NREM, REM bouts of all mice (pooled) across the three phenotypes for each condition: pre and post exposure to stress and phase: light and dark, in a dataframe structure compatible for pandas manipulation. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import glob
import os

In [ ]:
os.chdir('/Users/bmr225/Documents/PythonCodes/Survival_Data')

folders = ['Control_Dark_post','Control_Dark_pre','Control_Light_post','Control_Light_pre',
           'Resilient_Dark_post','Resilient_Dark_pre','Resilient_Light_post','Resilient_Light_pre',
           'Susceptible_Dark_post','Susceptible_Dark_pre','Susceptible_Light_post','Susceptible_Light_pre']


In [ ]:
def load_data(path, header):
    df = pd.read_excel(path, header=header)
    return df


In [ ]:
for folder in folders:
    main_direct = '/Users/bmr225/Documents/PythonCodes/'
    path = main_direct + folder+'/*.xlsx'
    
    #path = '/Users/bmr225/Documents/PythonCodes/Control_Dark_post/*.xlsx'

    folder_state_label = []
    folder_state_Num = []
    folder_Timestamp = []
    folder_Duration = []
    folder_Transition_to = []
    folder_Transition_from = []
    folder_Transition_to_label = []
    folder_Transition_from_label = []
    folder_Mouse_ID = []
    Mouse =0
     
        #Mouse = 0
    for filepath in glob.iglob(path):
        Mouse+=1
        print(filepath)
        data = load_data(filepath, None) # No header
        #data = load_data(path, None)
        data= data.iloc[:,[0,1,2,4,5,6,8,9,10]]
        data = data[2:] # Take the data less the header row
        
        NREM_timestamp = data[5].dropna().tolist()
        NREM_Duration = data[6].dropna().tolist()  
        NREM_num = [2]*len(NREM_Duration)
        NREM_cat = ['NREM']*len(NREM_Duration)
        #NREM.append(NREM_Duration)
        
        Wake_timestamp = data[1].dropna().tolist()
        Wake_Duration = data[2].dropna().tolist()
        Wake_cat = ['Wake']*len(Wake_Duration)
        Wake_num =[1]*len(Wake_Duration)
        
        
        #Wake.append(Wake_Duration)
        REM_timestamp = data[9].dropna().tolist()
        REM_Duration = data[10].dropna().tolist()
        REM_num = [4]*len(REM_Duration)
        REM_cat = ['REM']*len(REM_Duration)
        #REM.append(REM_Duration)
        # check
        #All_timestamp[:20]
        #All_timestamp[980:]
  
        All_timestamp = Wake_timestamp + NREM_timestamp + REM_timestamp
        All_duration = Wake_Duration + NREM_Duration + REM_Duration
        All_state_num = Wake_num+ NREM_num + REM_num
        All_state_cat = Wake_cat + NREM_cat + REM_cat
        tup_raw_data = list(zip(All_state_num,All_state_cat,All_timestamp,All_duration)) 
        tup_raw_data.sort(key = lambda x: x[2])   # sort it first
        
        # Sorting based on time steps and sequence of state occurrence
        # Now how to extract values from a list of tuples
        All_timestamp_sorted = [tup_raw_data[i][2] for i in range(len(tup_raw_data))]
       # assert vect == All_timestamp # won't work because it sorted
        All_state_num_sorted = [tup_raw_data[i][0] for i in range(len(tup_raw_data))]
        
        Transition_from_Num = [j - i for i, j in zip(All_state_num_sorted[: -1], All_state_num_sorted[1 :])] 
        Transition_to_Num = [j - i for i, j in zip(All_state_num_sorted[1:], All_state_num_sorted[: -1])]
        # Replacing numerical with labels/categories
        #Transition_to_cat = Transition_to_Num
        #Transition_from_cat = Transition_from_Num
        
        # Extract other lists from the tupple
        All_state_cat_sorted = [tup_raw_data[i][1] for i in range(len(tup_raw_data))]
        All_duration_sorted = [tup_raw_data[i][3] for i in range(len(tup_raw_data))]
        
        # Mapping numerical values to categorical values 
        # creating a dict file  
        # values where key matches 
    
        transitions_to_labels = {-2:'N-R',1:'N-W',-1:'W-N',3:'R-W',2:'R-N',-3:'W-R',0:'Glitch'}
        Transition_to_cat =[transitions_to_labels[item] for item in Transition_to_Num]
           
        transitions_from_labels = {2:'RfN',-1:'WfN',1:'NfW',-3:'WfR',-2:'NfR',3:'RfW',0:'Glitch'}
        Transition_from_cat =[transitions_from_labels[item] for item in Transition_from_Num]

        # Adjust the length of these 2/4 vectors:
        # For Transitions to: you have to append by the end because the last item is not done
        # we can put 'X' and '0' for transitions by the end
        Transition_to_cat.append('X')
        Transition_to_Num.append(0)

        # For the From: add to the begining
        Transition_from_cat.insert(0,'X')
        Transition_from_Num.insert(0,0)
        Mouse_ID = [Mouse]*len(All_timestamp_sorted)
        
        
      
        folder_state_label.append(All_state_cat_sorted,)
        folder_state_Num.append(All_state_num_sorted )
        folder_Timestamp.append(All_timestamp_sorted)
        folder_Duration.append(All_duration_sorted)
        folder_Transition_to.append(Transition_to_Num)
        folder_Transition_from .append(Transition_from_Num)
        folder_Transition_to_label.append(Transition_to_cat)
        folder_Transition_from_label.append(Transition_from_cat)
        folder_Mouse_ID.append(Mouse_ID)
         
    
    
    folder_state_label = sum(folder_state_label,[])
    folder_state_Num = sum(folder_state_Num,[])
    folder_Timestamp = sum(folder_Timestamp,[])
    folder_Duration = sum(folder_Duration,[])
    folder_Transition_to = sum(folder_Transition_to, [])
    folder_Transition_from = sum(folder_Transition_from,[])
    folder_Transition_to_label = sum(folder_Transition_to_label, [])
    folder_Transition_from_label = sum(folder_Transition_from_label,[])
    folder_Mouse_ID = sum(folder_Mouse_ID,[])
         
    

    
    data =  {'State_Label': folder_state_label,
             'State Number': folder_state_Num,
             'Timestamp': folder_Timestamp,
             'Duration': folder_Duration,
             'Transition_to': folder_Transition_to,
             'Transition_from': folder_Transition_from,
             'Transition_to_label': folder_Transition_to_label,
             'Transition_from_label': folder_Transition_from_label, # Most files end at 43200 Some files will require manual cleaning
             'Mouse ID': folder_Mouse_ID}
   
    
    data_df = pd.DataFrame({ key:pd.Series(value) for key, value in data.items() })


    filename = folder +'_grouped.csv'
    data_df.to_csv(filename,index = False)
    
    
    
    